# VOQC  <img src="umd.jpeg" alt="Drawing" style="width: 100px" align="right">


Welcome to VOQC, a verfied optimizer for quantum circuits! This tutorial is 

In [1]:
from qiskit import QuantumCircuit
from interop.voqc import SQIR
from interop.qiskit.voqc_optimization import VOQC
from qiskit.transpiler import PassManager
from qiskit.qasm import pi

## Passing a Qiskit Circuit Object to VOQC

Because of VOQC's interoperability with Cirq and Qiskit, it is now possible to pass a Qiskit circuit object through the VOQC optimizations and receive an optimized Qiskit circuit.

The VOQC transpiler can be called similarly to Qiskit's built-in transpiler passes (e.g. "Commutative Cancellation" and "CX Cancellation"). **VOQC(\[*list of optimizations*\])** can be appended to a pre-defined **Pass Manager** with "list of optimizations" being an optional argument used to run one or more of the unitary optiizations in VOQC (see *Running Individual Unitary Optimizations*). Appending **VOQC()**, without the argument, will run VOQC's primary optimization function, a specific order of the unitary optimizations. 

### Example 1

In our first example, we will pass a basic Qiskit circuit to VOQC without the optional list argument, taking advantage of VOQC's main optimization function.

We first create a four-qubit circuit with 2 NOT gates, 2 hadamard gates, and a CNOT gate. Given this sample circuit **circ**, we define a Pass Manager **pm** to schedule our VOQC transpiler pass. Then, we can append the VOQC transpiler pass to **pm** without an argument, **VOQC()**. Finally, we run our initial circuit **circ** through the primary VOQC optimization and receive and print our optimized circuit **new_circ**.  

We began with a circuit consisting of 5 gates, but have reduced it to just a singular gate. Now, we will go into more detail on the specifics of each individual optimization.


In [8]:
#Build Circuit with 4 Qubits and 5 Gates 
circ = QuantumCircuit(4)
circ.x(0)
circ.h(1)
circ.x(0)
circ.h(1)
circ.cx(2, 3)
print("Before Optimization:")
print(circ)

#Append VOQC pass without argument to the Pass Manager
pm = PassManager()
pm.append(VOQC())
new_circ = pm.run(circ)

#Print Optimized Circuit
print("After Optimization:")
print(new_circ)

Before Optimization:
     ┌───┐┌───┐
q_0: ┤ X ├┤ X ├
     ├───┤├───┤
q_1: ┤ H ├┤ H ├
     └───┘└───┘
q_2: ──■───────
     ┌─┴─┐     
q_3: ┤ X ├─────
     └───┘     
After Optimization:
          
q_0: ─────
          
q_1: ─────
          
q_2: ──■──
     ┌─┴─┐
q_3: ┤ X ├
     └───┘


## Running Individual Unitary Optimizations

While appending **VOQC()** to the pass manager runs the primary VOQC optimization function, there are scenarios where you may want to run one or more of the unitary optimizations of VOQC. These five optimizations are *not propagation*, *cancel single qubit gates*, *cancel two qubit gates*, *merge rotations*, and *hadamard reduction*. In this section, we will describe each of the five aforementioned optimizations, including their purpose and how to incorprate them into your compiler pass.

As always, the implementations of these optimizations can be found <a href="https://github.com/inQWIRE/SQIR">here</a> and the paper itself can be found <a href="https://arxiv.org/abs/1912.02250">here</a>. 

### Not Propagation

<p>The "not propagation" optimization cancels two NOT gates (X) that are either adjacent or separated by a circuit that commutes with X. For commuting circuits, the identities below are applied repeatedly to simplify the circuit.</p> 

<p><center>$X q;  H q \equiv H q;  Z q$</center></p>
<p><center>$X q;  Rz(k) q \approx Rz(2-k) q;  X q$</center></p>
<p><center>$X q_{1};  CNOT q_{1} q_{2} \equiv CNOT q_{1} q_{2};  X q_{1};  X q_{2}$</center></p>
<p><center>$X q_{2};  CNOT q_{1} q_{2} \equiv CNOT q_{1} q_{2};  X q_{2}$</center></p>


#### Example 2

This time, instead of using the transpiler pass without the optional list argument, we will pass a list of one element to VOQC. The optional list argument to invoke the *not propagation* optimization is **not_propagation**, so we append the following to the pass maanger: **VOQC(["not_propagation"])**. 
    
To test this, we will create a two-qubit circuit, consisting of 2 X gates and one CX gate. Once this is passed to VOQC, the leftmost X gate propagates through the CNOT gate, invoking the fourth identity above. Now, the circuit consists of a CNOT gate and two adjacent NOT gates. These adjacent X gates are "cancelled," leading to the final circuit, a single CNOT gate.




In [15]:
#Build Circuit with Two Qubits and Two Gates (CNOT and NOT)
circ = QuantumCircuit(2)
circ.x(1)
circ.cx(0, 1)
circ.x(1)
print("Before Optimization:")
print(circ)

#Append "not_propagation" optimization to the Pass Manager
pm = PassManager()
pm.append(VOQC(["not_propagation"]))
new_circ = pm.run(circ)

#Print Optimized Circuit
print("After Optimization:")
print(new_circ)

Before Optimization:
                    
q_0: ───────■───────
     ┌───┐┌─┴─┐┌───┐
q_1: ┤ X ├┤ X ├┤ X ├
     └───┘└───┘└───┘
After Optimization:
          
q_0: ──■──
     ┌─┴─┐
q_1: ┤ X ├
     └───┘


### Cancel Single Qubit Gates

In addition to *not propgation*, **cancel single qubit gates** is another *propagate-cancel* optimization in VOQC. This also uses cancellation and commutation rules. The cancellation rules are based upon the fact that X and H gates are self-cancelling and the commutation rules can be seen in *Figure 6* of Hietala et al. [2019]. Gowever, a key difference between this cancellation and *not_propagation* is that the gates revert back to their original positons if they cannot commute or cancel. 

Here is an example of the self-cancelling nature of an X gate. Multiplying these matrices results in the identity matrix.

<p><center>$\begin{bmatrix}0 & 1\\1 & 0\end{bmatrix}$
$\begin{bmatrix}0 & 1\\1 & 0\end{bmatrix}$
= $\begin{bmatrix}1 & 0\\0 & 1\end{bmatrix}$</center></p>

#### Example 3

Similarly to **not_propagation**, we will append the **VOQC(["cancel_single_qubit_gates"])** to the pass manager to call the "cancel_single_qubit_gates" function in OCaml. 
    
We define a single qubit circuit **circ** with three gates and take advantage of the fact that X gates are self-cancelling. Therefore, we expect that we will end up with a single gate, a z-axis rotation of $\frac {\pi}{2}$. This rotation is a special case rotation, simplifying to an S gate. 




In [10]:
#Build Circuit with One Qubit and Three Gates (Rz and 2 Hadamard)
circ = QuantumCircuit(1)
circ.rz(pi/2, 0)
circ.h(0)
circ.h(0)
print("Before Optimization:")
print(circ)

#Append "cancel_single_qubit_gates" optimization to the Pass Manager
pm = PassManager()
pm.append(VOQC(["cancel_single_qubit_gates"]))
new_circ = pm.run(circ)

#Print Optimized Circuit
print("After Optimization:")
print(new_circ)

Before Optimization:
     ┌──────────┐┌───┐┌───┐
q_0: ┤ RZ(pi/2) ├┤ H ├┤ H ├
     └──────────┘└───┘└───┘
After Optimization:
     ┌───┐
q_0: ┤ S ├
     └───┘


### Cancel Two Qubit Gates

This optimization is almost identical to the *cancel_single_qubit_gates* optimization in the previous section, except for the fact that it applies to two-qubit gates (CX), rather than the X, H, and Rz gates. 

<p><center>$\begin{bmatrix}1 & 0 & 0 & 0\\0 & 1 & 0 & 0\\0 & 0 & 0 & 1\\0 & 0 & 1 & 0\end{bmatrix}$
$\begin{bmatrix}1 & 0 & 0 & 0\\0 & 1 & 0 & 0\\0 & 0 & 0 & 1\\0 & 0 & 1 & 0\end{bmatrix}$
= $\begin{bmatrix}1 & 0 & 0 & 0\\0 & 1 & 0 & 0\\0 & 0 & 1 & 0\\0 & 0 & 0 & 1\end{bmatrix}$</center></p>

#### Example 4

In our new circuit, we will have an H gate act on qubit 0 and two CNOT gates with control qubit 0 and target qubit 1. Just like earlier, we will add **VOQC(["cancel_two_qubit_gates"])** to the pass manager and run the circuit. 
    
The two CX gates cancel and the final circuit contains just the original hadamard gate.



In [11]:
#Build Circuit with Two Qubits and Three Gates (2 CNOT and Hadamard)
circ = QuantumCircuit(2)
circ.cx(0, 1)
circ.cx(0, 1)
circ.h(0)
print("Before Optimization:")
print(circ)

#Append "cancel_two_qubit_gates" optimization to the Pass Manager
pm = PassManager()
pm.append(VOQC(["cancel_two_qubit_gates"]))
new_circ = pm.run(circ)

#Print Optimized Circuit
print("After Optimization:")
print(new_circ)

Before Optimization:
               ┌───┐
q_0: ──■────■──┤ H ├
     ┌─┴─┐┌─┴─┐└───┘
q_1: ┤ X ├┤ X ├─────
     └───┘└───┘     
After Optimization:
     ┌───┐
q_0: ┤ H ├
     └───┘
q_1: ─────
          


### Merge Rotations

#### Example 5

This time, instead of using the transpiler pass without the optional list argument, we will pass a list of one element to VOQC. The optional list argument to invoke the *not propagation* optimization is **not_propagation**, so we append the following to the pass maanger: **VOQC(["not_propagation"])**. 
    
To test this, we will create a two-qubit circuit, consisting of 2 X gates and one CX gate. Once this is passed to VOQC, the leftmost X gate propagates through the CNOT gate, invoking the fourth identity above. Now, the circuit consists of a CNOT gate and two adjacent NOT gates. These adjacent X gates are "cancelled," leading to the final circuit, a single CNOT gate.




In [13]:
#Build Circuit with One Qubit and Two Gates (Two Rz)
circ = QuantumCircuit(1)
circ.rz(pi/4, 0)
circ.rz(pi/4, 0)
print("Before Optimization:")
print(circ)

#Append "merge_rotations" optimization to the Pass Manager
pm = PassManager()
pm.append(VOQC(["merge_rotations"]))
new_circ = pm.run(circ)

#Print Optimized Circuit
print("After Optimization:")
print(new_circ)

Before Optimization:
     ┌──────────┐┌──────────┐
q_0: ┤ RZ(pi/4) ├┤ RZ(pi/4) ├
     └──────────┘└──────────┘
After Optimization:
     ┌───┐
q_0: ┤ S ├
     └───┘


### Hadamard Reduction

#### Example 6

This time, instead of using the transpiler pass without the optional list argument, we will pass a list of one element to VOQC. The optional list argument to invoke the *not propagation* optimization is **not_propagation**, so we append the following to the pass maanger: **VOQC(["not_propagation"])**. 
    
To test this, we will create a two-qubit circuit, consisting of 2 X gates and one CX gate. Once this is passed to VOQC, the leftmost X gate propagates through the CNOT gate, invoking the fourth identity above. Now, the circuit consists of a CNOT gate and two adjacent NOT gates. These adjacent X gates are "cancelled," leading to the final circuit, a single CNOT gate.




In [14]:
#Build Circuit with Two Qubits and Two Gates (CNOT and NOT)
circ = QuantumCircuit(1)
circ.h(0)
circ.rz(pi/2, 0)
circ.h(0)

print("Before Optimization:")
print(circ)

#Append "hadamard_reduction" optimization to the Pass Manager
pm = PassManager()
pm.append(VOQC(["hadamard_reduction"]))
new_circ = pm.run(circ)

#Print Optimized Circuit
print("After Optimization:")
print(new_circ)

Before Optimization:
     ┌───┐┌──────────┐┌───┐
q_0: ┤ H ├┤ RZ(pi/2) ├┤ H ├
     └───┘└──────────┘└───┘
After Optimization:
     ┌─────┐┌───┐┌─────┐
q_0: ┤ SDG ├┤ H ├┤ SDG ├
     └─────┘└───┘└─────┘
